In [29]:
import sys
import cv2
import numpy as np
import math
import imutils
from skimage.filters import threshold_local

def call_back_func(event, x, y, flags, pixel_location):
    #Gives us location of pixel when clicked
    if event == cv2.EVENT_LBUTTONDOWN:
        pixel_location.append([x,y])
        
def read_file(image_path):
    img = cv2.imread(image_path)
    img = imutils.resize(img, height=700)
    
    return img

def select_pixel_location(img):
    pixel_location = []
    
    org = [img.shape[1]//3, img.shape[0]//2]
    text = ["Click Top Left corner", "Click Top Right corner","Click Bottom Right corner","Click Bottom Left corner"]
    while (len(pixel_location) < 4):
        index = len(pixel_location)
        clone = img.copy()
        
        font = cv2.FONT_HERSHEY_SIMPLEX
  
        #org = (50, 50)
        fontScale = 0.55
        color = (0, 0, 255)
        thickness = 2
        
        clone = cv2.putText(clone, text[index], org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        
        cv2.namedWindow("Original_Image")
        cv2.imshow("Original_Image", clone)
        cv2.setMouseCallback("Original_Image", call_back_func, pixel_location)
        
        k = cv2.waitKey(1)
        if cv2.getWindowProperty('Original_Image', cv2.WND_PROP_VISIBLE) < 1:
            
            return 0

    cv2.destroyAllWindows()

    return pixel_location

def generate_homography(pixel_loc, final_loc):
    #Generates A matrix based on point co-ordinates from images
    A = []
    for i in range(len(pixel_loc)):
        l1 = [0, 0, 0,                             -1*pixel_loc[i][1], -1*pixel_loc[i][0], -1*1,      final_loc[i][1]*pixel_loc[i][1], final_loc[i][1]*pixel_loc[i][0], final_loc[i][1]*1]
        l2 = [1*pixel_loc[i][1], 1*pixel_loc[i][0], 1*1,              0,0,0,                         -final_loc[i][0]*pixel_loc[i][1], -final_loc[i][0]*pixel_loc[i][0], -final_loc[i][0]*1]
        A.append(l1)
        A.append(l2)
    A = np.array(A)
    return A

def calculate_svd(A):
    #Calculating SVD and taking the last column as it has the smallest eigen value. Converting it to 3x3 matrix
    U, D, V = np.linalg.svd(A)
    H = np.reshape(V[-1], (3,3))
    return H    

def apply_Bilinear_Interpolation(pull_cord, img):
    i, j = int(pull_cord[0]), int(pull_cord[1])
    a, b = pull_cord[0] - i, pull_cord[1] - j

    r = (1-a)*(1-b)*img[i,j,0]  +  a*(1-b)*img[i+1,j,0]  +  a*b*img[i+1,j+1,0]  +  (1-a)*b*img[i,j+1,0]
    g = (1-a)*(1-b)*img[i,j,1]  +  a*(1-b)*img[i+1,j,1]  +  a*b*img[i+1,j+1,1]  +  (1-a)*b*img[i,j+1,1]
    b = (1-a)*(1-b)*img[i,j,2]  +  a*(1-b)*img[i+1,j,2]  +  a*b*img[i+1,j+1,2]  +  (1-a)*b*img[i,j+1,2]
    return np.array([r,g,b])
    
def generate_Inverse_Warp_final_image(H, img, final_dim, name):
    #Using Inverse warping technique with and without Bilinear Interpolation to generate image
    final_image = np.zeros(final_dim)
    final_image_with_BLI = np.zeros(final_dim)
    
    H_inv = np.linalg.inv(H)
    for x in range(final_image.shape[0]):
        for y in range(final_image.shape[1]):
            final_cord = np.array([x,y,1])
            pull_cord = np.matmul(H_inv, final_cord)
            pull_cord[0] = pull_cord[0]/pull_cord[2]
            pull_cord[1] = pull_cord[1]/pull_cord[2]
            if (0 <= pull_cord[0] < img.shape[0]) and (0 <= pull_cord[1] < img.shape[1]):
                final_image[x,y] = img[int(pull_cord[0]), int(pull_cord[1])]
                final_image_with_BLI[x,y] = apply_Bilinear_Interpolation(pull_cord, img)
    return final_image_with_BLI
        
   

In [32]:
final_dim = (940, 500, 3)
#Final locations on the transformed image
final_loc = [[1, 1], [1, final_dim[1]], list(final_dim[:2]), [final_dim[0], 1]]

image_path = "Input_Images/input.jpg"
img = read_file(image_path)
print("Please check if the image window is minimized")
print("Pixel location provided in code IF NOT ABLE TO SELECT  PIXEL FROM IMAGE")
print("Opening Image Window to select 4 corners of the field\n Order:\n1.Top Left\n2.Top Right\n3.Bottom Right\n4.Bottom Left\n")


####  UNCOMMENT THIS IF NOT ABLE TO SELECT  PIXEL FROM IMAGE #####
#pixel_loc = [[246, 51], [403, 73], [283, 277], [21, 192]]

pixel_loc = select_pixel_location(img)
if pixel_loc == 0:
    print("Window Closed before selecting 4 points.")
    sys.exit(0)
A = generate_homography(pixel_loc, final_loc)
H = calculate_svd(A)

print("Generating Inverse Warp With BLI Image With Points")
warped = generate_Inverse_Warp_final_image(H, img, final_dim, "Points")
print("Warped shape", warped.shape)
warped = cv2.cvtColor(warped.astype("uint8"), cv2.COLOR_BGR2GRAY)
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255

cv2.imwrite(".\\Output_Images\\Inverse_Warp_BLI.jpeg", warped)
print("Output Image created in Output Directory")


Please check if the image window is minimized
Pixel location provided in code IF NOT ABLE TO SELECT  PIXEL FROM IMAGE
Opening Image Window to select 4 corners of the field
 Order:
1.Top Left
2.Top Right
3.Bottom Right
4.Bottom Left

Generating Inverse Warp With BLI Image With Points
Warped shape (940, 500, 3)
Output Image created in Output Directory
